https://colab.research.google.com/drive/1id7gcj7QMF35lPQHaHtTEg52tx_zVep6#scrollTo=kcOy9dq3NlAi

In [ ]:
# Idioma escolhido
language = 'pt'

In [ ]:
# Instalação de dependências
!pip install git+https://github.com/openai/whisper.git -q
!pip install gTTS playsound openai -q
!apt-get install ffmpeg -y

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


# 1. Gravação de Áudio Com Python (e Uma Pitada de JavaScript) 🎤

In [ ]:
# ============================================
# PROTÓTIPO DE ACESSIBILIDADE (TESTE EM COLAB)
# ============================================
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode
import os
import openai
import whisper
from gtts import gTTS
import playsound

# ------------------------------- # 1. Gravação de áudio (Colab + JS)
# -------------------------------
RECORD = """
  const sleep = time => new Promise(resolve => setTimeout(resolve, time))
  const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
def record(sec=5):
    display(Javascript(RECORD))
    js_result = output.eval_js('record(%s)' % (sec * 1000))
    audio = b64decode(js_result.split(',')[1])
    file_name = 'request_audio.wav'
    with open(file_name, 'wb') as f:
        f.write(audio)
    return f'/content/{file_name}'

print("Ouvindo...\n")
record_file = record()
display(Audio(record_file, autoplay=False))

Ouvindo...



<IPython.core.display.Javascript object>

# 2. Reconhecimento de Fala com Whisper (OpenAI) 🧠

In [46]:
import whisper
# -------------------------------
# 2. Transcrição com Whisper
# -------------------------------
model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language="pt")
transcription = result["text"]
print("Transcrição:", transcription)

Transcrição:  Qual o nome do criador do avião?


# 3. Integração com a API do ChatGPT 💬

In [ ]:
 # -------------------------------
# 3. Integração com ChatGPT - (com fallback)
# -------------------------------

from openai import OpenAI
import os

# Configure sua chave (se tiver créditos ativos)
#Troque o campo "SUA_CHAVE_AQUI" pela sua chave que será criada no site da OpenAI(OPENAI_API_KEY)
os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_AQUI"
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

print("Pergunta transcrita:", transcription)

try:
    # Tentativa de chamada real à API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": transcription}
        ]
    )
    chatgpt_response = response.choices[0].message.content

except Exception as e:
    # Fallback: resposta simulada para garantir que o fluxo  funcione mesmo sem créditos.
    print("⚠️ Erro na API:", e)
    chatgpt_response = "Esta é uma resposta simulada para teste do fluxo."

print("Resposta do ChatGPT:", chatgpt_response)

Pergunta transcrita:  Qual o nome do criador do avião?
⚠️ Erro na API: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Resposta do ChatGPT: Esta é uma resposta simulada para teste do fluxo.


# 4. Sintetizando a Resposta do ChatGPT Como Voz (gTTS) 🔊

In [48]:
# -------------------------------
# 4. Síntese de voz com gTTS
# -------------------------------
from gtts import gTTS
import playsound

# Gera o arquivo de áudio
tts = gTTS(chatgpt_response, lang="pt")
tts.save("response.mp3")

# Exibe player de áudio no Colab
Audio("response.mp3")